<a href="https://colab.research.google.com/github/Nomaqui/ENTREGAS/blob/main/02%20-%20preprocesado.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Leer el contenido, y cargar las librerias que vamos a usar


In [1]:
import pandas as pd
from sklearn.impute import SimpleImputer

df = pd.read_csv('train.csv')

# Guardar columna objetivo y ID
y = df['RENDIMIENTO_GLOBAL']

X = df.drop(columns=['RENDIMIENTO_GLOBAL', 'ID'])


Mostrar el contenido en una tabla


In [38]:
df

,ID,PERIODO,ESTU_PRGM_ACADEMICO,ESTU_PRGM_DEPARTAMENTO,ESTU_VALORMATRICULAUNIVERSIDAD,ESTU_HORASSEMANATRABAJA,FAMI_ESTRATOVIVIENDA,FAMI_TIENEINTERNET,FAMI_EDUCACIONPADRE,FAMI_TIENELAVADORA,...,ESTU_PRIVADO_LIBERTAD,ESTU_PAGOMATRICULAPROPIO,FAMI_TIENECOMPUTADOR,FAMI_TIENEINTERNET.1,FAMI_EDUCACIONMADRE,RENDIMIENTO_GLOBAL,coef_1,coef_2,coef_3,coef_4
0,904256,20212,ENFERMERIA,BOGOTÁ,Entre 5.5 millones y menos de 7 millones,Menos de 10 horas,Estrato 3,Si,Técnica o tecnológica incompleta,Si,...,N,No,Si,Si,Postgrado,medio-alto,0.322,0.208,0.310,0.267
1,645256,20212,DERECHO,ATLANTICO,Entre 2.5 millones y menos de 4 millones,0,Estrato 3,No,Técnica o tecnológica completa,Si,...,N,No,Si,No,Técnica o tecnológica incompleta,bajo,0.311,0.215,0.292,0.264
2,308367,20203,MERCADEO Y PUBLICIDAD,BOGOTÁ,Entre 2.5 millones y menos de 4 millones,Más de 30 horas,Estrato 3,Si,Secundaria (Bachillerato) completa,Si,...,N,No,No,Si,Secundaria (Bachillerato) completa,bajo,0.297,0.214,0.305,0.264
3,470353,20195,ADMINISTRACION DE EMPRESAS,SANTANDER,Entre 4 millones y menos de 5.5 millones,0,Estrato 4,Si,No sabe,Si,...,N,No,Si,Si,Secundaria (Bachillerato) completa,alto,0.485,0.172,0.252,0.190
4,989032,20212,PSICOLOGIA,ANTIOQUIA,Entre 2.5 millones y menos de 4 millones,Entre 21 y 30 horas,Estrato 3,Si,Primaria completa,Si,...,N,No,Si,Si,Primaria completa,medio-bajo,0.316,0.232,0.285,0.294
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
692495,25096,20195,BIOLOGIA,LA GUAJIRA,Entre 500 mil y menos de 1 millón,Entre 11 y 20 horas,Estrato 2,Si,Secundaria (Bachillerato) completa,Si,...,N,Si,Si,Si,Secundaria (Bachillerato) incompleta,medio-alto,0.237,0.271,0.271,0.311
692496,754213,20212,PSICOLOGIA,NORTE SANTANDER,Entre 2.5 millones y menos de 4 millones,Más de 30 horas,Estrato 3,Si,Primaria incompleta,Si,...,N,No,Si,Si,Secundaria (Bachillerato) incompleta,bajo,0.314,0.240,0.278,0.260
692497,504185,20183,ADMINISTRACIÓN EN SALUD OCUPACIONAL,BOGOTÁ,Entre 1 millón y menos de 2.5 millones,Menos de 10 horas,Estrato 3,Si,Secundaria (Bachillerato) completa,Si,...,N,Si,Si,Si,Secundaria (Bachillerato) incompleta,medio-bajo,0.286,0.240,0.314,0.287
692498,986620,20195,PSICOLOGIA,TOLIMA,Entre 2.5 millones y menos de 4 millones,Menos de 10 horas,Estrato 1,No,Primaria completa,No,...,N,Si,Si,No,Primaria completa,bajo,0.132,0.426,0.261,0.328


Separar columnas por tipo

In [2]:
cat_cols = X.select_dtypes(include=['object', 'category']).columns.tolist()
num_cols = X.select_dtypes(include=['int64', 'float64']).columns.tolist()

print("Categóricas:", cat_cols)
print("Numéricas:", num_cols)

Categóricas: ['ESTU_PRGM_ACADEMICO', 'ESTU_PRGM_DEPARTAMENTO', 'ESTU_VALORMATRICULAUNIVERSIDAD', 'ESTU_HORASSEMANATRABAJA', 'FAMI_ESTRATOVIVIENDA', 'FAMI_TIENEINTERNET', 'FAMI_EDUCACIONPADRE', 'FAMI_TIENELAVADORA', 'FAMI_TIENEAUTOMOVIL', 'ESTU_PRIVADO_LIBERTAD', 'ESTU_PAGOMATRICULAPROPIO', 'FAMI_TIENECOMPUTADOR', 'FAMI_TIENEINTERNET.1', 'FAMI_EDUCACIONMADRE']
Numéricas: ['PERIODO', 'coef_1', 'coef_2', 'coef_3', 'coef_4']


Imputar numéricas con la media

In [3]:
imp_num = SimpleImputer(strategy='mean')
X[num_cols] = imp_num.fit_transform(X[num_cols])

Imputar categóricas con la moda

In [4]:
imp_cat = SimpleImputer(strategy='most_frequent')
X[cat_cols] = imp_cat.fit_transform(X[cat_cols])

Mostrar la tabla con los datos imputados

In [21]:
X

,PERIODO,ESTU_PRGM_ACADEMICO,ESTU_PRGM_DEPARTAMENTO,ESTU_VALORMATRICULAUNIVERSIDAD,ESTU_HORASSEMANATRABAJA,FAMI_ESTRATOVIVIENDA,FAMI_TIENEINTERNET,FAMI_EDUCACIONPADRE,FAMI_TIENELAVADORA,FAMI_TIENEAUTOMOVIL,ESTU_PRIVADO_LIBERTAD,ESTU_PAGOMATRICULAPROPIO,FAMI_TIENECOMPUTADOR,FAMI_TIENEINTERNET.1,FAMI_EDUCACIONMADRE,coef_1,coef_2,coef_3,coef_4
0,20212.0,ENFERMERIA,BOGOTÁ,Entre 5.5 millones y menos de 7 millones,Menos de 10 horas,Estrato 3,Si,Técnica o tecnológica incompleta,Si,Si,N,No,Si,Si,Postgrado,0.322,0.208,0.310,0.267
1,20212.0,DERECHO,ATLANTICO,Entre 2.5 millones y menos de 4 millones,0,Estrato 3,No,Técnica o tecnológica completa,Si,No,N,No,Si,No,Técnica o tecnológica incompleta,0.311,0.215,0.292,0.264
2,20203.0,MERCADEO Y PUBLICIDAD,BOGOTÁ,Entre 2.5 millones y menos de 4 millones,Más de 30 horas,Estrato 3,Si,Secundaria (Bachillerato) completa,Si,No,N,No,No,Si,Secundaria (Bachillerato) completa,0.297,0.214,0.305,0.264
3,20195.0,ADMINISTRACION DE EMPRESAS,SANTANDER,Entre 4 millones y menos de 5.5 millones,0,Estrato 4,Si,No sabe,Si,No,N,No,Si,Si,Secundaria (Bachillerato) completa,0.485,0.172,0.252,0.190
4,20212.0,PSICOLOGIA,ANTIOQUIA,Entre 2.5 millones y menos de 4 millones,Entre 21 y 30 horas,Estrato 3,Si,Primaria completa,Si,Si,N,No,Si,Si,Primaria completa,0.316,0.232,0.285,0.294
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
692495,20195.0,BIOLOGIA,LA GUAJIRA,Entre 500 mil y menos de 1 millón,Entre 11 y 20 horas,Estrato 2,Si,Secundaria (Bachillerato) completa,Si,No,N,Si,Si,Si,Secundaria (Bachillerato) incompleta,0.237,0.271,0.271,0.311
692496,20212.0,PSICOLOGIA,NORTE SANTANDER,Entre 2.5 millones y menos de 4 millones,Más de 30 horas,Estrato 3,Si,Primaria incompleta,Si,No,N,No,Si,Si,Secundaria (Bachillerato) incompleta,0.314,0.240,0.278,0.260
692497,20183.0,ADMINISTRACIÓN EN SALUD OCUPACIONAL,BOGOTÁ,Entre 1 millón y menos de 2.5 millones,Menos de 10 horas,Estrato 3,Si,Secundaria (Bachillerato) completa,Si,No,N,Si,Si,Si,Secundaria (Bachillerato) incompleta,0.286,0.240,0.314,0.287
692498,20195.0,PSICOLOGIA,TOLIMA,Entre 2.5 millones y menos de 4 millones,Menos de 10 horas,Estrato 1,No,Primaria completa,No,No,N,Si,Si,No,Primaria completa,0.132,0.426,0.261,0.328


Convertir columnas en one-hot

In [5]:
X_convertida = pd.get_dummies(X, columns=cat_cols, drop_first=True)

Mostrar la tabla con las columnas debidas en one-hot

In [23]:
X_convertida

,PERIODO,coef_1,coef_2,coef_3,coef_4,ESTU_PRGM_ACADEMICO_ACTIVIDAD FISICA Y DEPORTE,ESTU_PRGM_ACADEMICO_ACUICULTURA,ESTU_PRGM_ACADEMICO_ADMINISTRACION,ESTU_PRGM_ACADEMICO_ADMINISTRACION FINANCIERA,ESTU_PRGM_ACADEMICO_ADMINISTRACION & SERVICIO,...,FAMI_EDUCACIONMADRE_Ninguno,FAMI_EDUCACIONMADRE_No Aplica,FAMI_EDUCACIONMADRE_No sabe,FAMI_EDUCACIONMADRE_Postgrado,FAMI_EDUCACIONMADRE_Primaria completa,FAMI_EDUCACIONMADRE_Primaria incompleta,FAMI_EDUCACIONMADRE_Secundaria (Bachillerato) completa,FAMI_EDUCACIONMADRE_Secundaria (Bachillerato) incompleta,FAMI_EDUCACIONMADRE_Técnica o tecnológica completa,FAMI_EDUCACIONMADRE_Técnica o tecnológica incompleta
0,20212.0,0.322,0.208,0.310,0.267,False,False,False,False,False,...,False,False,False,True,False,False,False,False,False,False
1,20212.0,0.311,0.215,0.292,0.264,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,True
2,20203.0,0.297,0.214,0.305,0.264,False,False,False,False,False,...,False,False,False,False,False,False,True,False,False,False
3,20195.0,0.485,0.172,0.252,0.190,False,False,False,False,False,...,False,False,False,False,False,False,True,False,False,False
4,20212.0,0.316,0.232,0.285,0.294,False,False,False,False,False,...,False,False,False,False,True,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
692495,20195.0,0.237,0.271,0.271,0.311,False,False,False,False,False,...,False,False,False,False,False,False,False,True,False,False
692496,20212.0,0.314,0.240,0.278,0.260,False,False,False,False,False,...,False,False,False,False,False,False,False,True,False,False
692497,20183.0,0.286,0.240,0.314,0.287,False,False,False,False,False,...,False,False,False,False,False,False,False,True,False,False
692498,20195.0,0.132,0.426,0.261,0.328,False,False,False,False,False,...,False,False,False,False,True,False,False,False,False,False


In [24]:
y.value_counts()

,count
RENDIMIENTO_GLOBAL,
alto,175619
bajo,172987
medio-bajo,172275
medio-alto,171619


In [15]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test= train_test_split(X_convertida, y,  test_size=0.42857184, random_state=19, stratify=y)
clf = RandomForestClassifier(n_estimators=100,  random_state=19)

clf.fit(X_train, y_train)

RandomForestClassifier(random_state=19)

In [16]:
from sklearn.metrics import classification_report, confusion_matrix

y_pred = clf.predict(X_test)

print(classification_report(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))

              precision    recall  f1-score   support

        alto       0.52      0.61      0.56     75266
        bajo       0.42      0.51      0.46     74137
  medio-alto       0.31      0.24      0.27     73551
  medio-bajo       0.31      0.26      0.28     73832

    accuracy                           0.41    296786
   macro avg       0.39      0.40      0.39    296786
weighted avg       0.39      0.41      0.39    296786

[[45593  7757 13177  8739]
 [ 7779 37616 11177 17565]
 [22123 17202 17861 16365]
 [12781 26396 15458 19197]]


In [18]:
df_test = pd.read_csv("test.csv")
ids_test = df_test["ID"]
X_test = df_test.drop(columns=["ID"])

In [19]:
# Imputar valores faltantes
X_test[num_cols] = imp_num.transform(X_test[num_cols])
X_test[cat_cols] = imp_cat.transform(X_test[cat_cols])

# Codificación one-hot con las mismas columnas que el entrenamiento
X_test_convertida = pd.get_dummies(X_test, columns=cat_cols, drop_first=True)

# Asegurar que tenga las mismas columnas que X_convertida
X_test_convertida = X_test_convertida.reindex(columns=X_convertida.columns, fill_value=0)

In [20]:
y_test_pred = clf.predict(X_test_convertida)

In [21]:
print(len(y_test_pred))

296786


In [22]:
submission = pd.DataFrame({
    "ID": ids_test,
    "RENDIMIENTO_GLOBAL": y_test_pred
})

In [23]:
submission.to_csv("submission.csv", index=False)
